# PEFT tutorial using Hugging Face
## 教學目標
利用 Hugging Face 套件快速使用 PEFT 來進行下游任務訓練
- 單一句型分類任務 (single-sentence text classification)

## 適用對象
已經有基本的機器學習知識，且擁有 Python、`numpy`、`pandas`、`scikit-learn` 以及 `PyTorch` 基礎的學生。

若沒有先學過 Python，請參考 [python-入門語法](./python-入門語法.ipynb) 教學。

若沒有先學過 `pandas`，請參考 [pandas-基本功能](./pandas-基本功能.ipynb) 教學。

若沒有先學過 `numpy`，請參考 [numpy-基本功能](./numpy-基本功能.ipynb) 教學。

若沒有先學過 `scikit-learn`，請參考 [scikit-learn-基本功能](./scikit-learn-基本功能.ipynb) 教學。

若沒有先學過  `PyTorch` ，請參考 [PyTorch-基本功能](./PyTorch-基本功能.ipynb) 教學。

若沒有先學過如何使用 `PyTorch` 建立自然語言處理序列模型，請參考 [NN-中文文本分類](./NN-中文文本分類.ipynb) 教學。

## PEFT 簡易介紹
### 對大語言模型進行微調的挑戰
- 大語言模型的通常是以大量的文本資料進行訓練，並且在多個任務上取得了驚人的表現。
- 若我們想要將這些大語言模型應用在自己的任務上，通常需要進行微調。
- 但是對於大語言模型進行微調是一個挑戰，因為這些模型通常有數十億甚至數百億的參數，並且需要大量的計算資源。
- 這就是為什麼我們需要 PEFT 這個套件，它可以幫助我們快速的進行大語言模型的微調。
![](https://i.imgur.com/q6u4GVJ.png)
- 更多細節請參考 ([Peft github](https://github.com/huggingface/peft))

## PEFT 範例: LoRA
![](https://i.imgur.com/GCsNYXF.png)
- 請參考理論層面的詳細教學 ([影片連結](https://www.youtube.com/watch?v=dA-NhCtrrVE))
- 也可以參考原始論文 ([論文連結](https://arxiv.org/abs/2106.09685))

## Hugging Face 介紹
- 🤗 Hugging Face 是專門提供自然語言處理領域的函式庫
- 其函式庫支援 PyTorch 和 TensorFlow
- 🤗 Hugging Face 的主要套件為:
    1. Transformers ([連結](https://huggingface.co/transformers/index.html))
    - 提供了現今最強大的自然語言處理模型，使用上非常彈性且方便
    2. Tokenizers ([連結](https://huggingface.co/docs/tokenizers/python/latest/))
    - 讓你可以快速做好 BERT 系列模型 tokenization
    3. Datasets ([連結](https://huggingface.co/docs/datasets/))
    - 提供多種自然語言處理任務的資料集

In [ ]:
# 若沒有安裝 transformers 和 datasets 套件，請取消以下註解並執行
!pip install transformers==4.38.0
!pip install datasets
!pip install torch==2.0.1+cu110
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 17.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggi

In [ ]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -r requirements.txt
!pip install -v --disable-pip-version-check --no-cache-dir ./

Cloning into 'apex'...
remote: Enumerating objects: 11703, done.
remote: Counting objects: 100% (3771/3771), done.
remote: Compressing objects: 100% (618/618), done.
remote: Total 11703 (delta 3381), reused 3298 (delta 3149), pack-reused 7932
Receiving objects: 100% (11703/11703), 15.53 MiB | 18.49 MiB/s, done.
Resolving deltas: 100% (8210/8210), done.
/content/apex
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Processing /content/apex
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached setuptools-69.5.1-py3-none-any.whl (894 kB)
    Using cached wheel-0.43.0-py3-none-any.whl (65 kB)
    Creating /tmp/pip-build-env-o28eecc1/overlay/local/bin
    changing mode of /tmp/pip-build-env-o28eecc1/overlay/local/bin/wheel to 755
  ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is

In [ ]:
# 1. 確認所需套件的版本
import torch
print("PyTorch 的版本為: {}".format(torch.__version__))

import transformers
print("Hugging Face Transformers 的版本為: {}".format(transformers.__version__))

import datasets
print("Hugging Face Datasets 的版本為: {}".format(datasets.__version__))

import peft
print("PEFT 的版本為: {}".format(peft.__version__))

PyTorch 的版本為: 2.2.1+cu121
Hugging Face Transformers 的版本為: 4.38.0
Hugging Face Datasets 的版本為: 2.19.0
PEFT 的版本為: 0.10.0


In [ ]:
# 2. 載入其他所需套件

import os
import json
import numpy as np
from pathlib import Path # (Python3.4+)

# 單一句型分類任務 (single-sentence text classification)
## 準備資料集 (需先下載)
我們使用 IMDb reviews 資料集作為範例

In [ ]:
# 若沒有安裝 wget 套件，請取消以下註解並執行
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=dc79573e67a9b06435524ef0eaf6b42a29df72c6258aab36050602cd52bc6b6e
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
# 3. 下載 IMDb 資料集
# import wget
# url = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
# filename = wget.download(url, out='./')

In [ ]:
from datasets import load_dataset

dataset1 = load_dataset("glue", "sst2")
dataset2 = load_dataset("glue", "qnli")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/104743 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5463 [00:00<?, ? examples/s]

In [ ]:
print(type(dataset1))
print(type(dataset2))


<class 'datasets.dataset_dict.DatasetDict'>
<class 'datasets.dataset_dict.DatasetDict'>


In [ ]:
# # 若沒有安裝 tarfile 套件，請取消以下註解並執行
# !pip install tarfile

In [ ]:
# # 4. 解壓縮 IMDb 資料集
# import tarfile

# # 指定檔案位置，並解壓縮 .gz 結尾的壓縮檔
# tar = tarfile.open('aclImdb_v1.tar.gz', 'r:gz')
# tar.extractall()

In [ ]:
# 以上是一種

## 接下來我們要進行資料前處理
但首先要觀察解壓縮後的資料夾結構:
```
aclImdb---
        |--train
        |    |--neg
        |    |--pos
        |    |--...
        |--test
        |    |--neg
        |    |--pos
        |    |--...
        |--imdb.vocab
        |--imdbEr.text
        |--README
```
其中 train 和 test 資料夾中分別又有 neg 和 pos 兩種資料夾

我們要針對這兩個目標資料夾進行處理

In [ ]:
# 获取所有的键
keys1 = dataset1.keys()

# 遍历所有的键，并查看对应的数据集结构
for key in keys1:
    print(f"Dataset: {key}")
    print(dataset1[key])
    print("\n")


Dataset: train
Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
})


Dataset: validation
Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 872
})


Dataset: test
Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 1821
})




In [ ]:
# 获取所有的键
keys2 = dataset2.keys()

# 遍历所有的键，并查看对应的数据集结构
for key in keys2:
    print(f"Dataset: {key}")
    print(dataset2[key])
    print("\n")

Dataset: train
Dataset({
    features: ['question', 'sentence', 'label', 'idx'],
    num_rows: 104743
})


Dataset: validation
Dataset({
    features: ['question', 'sentence', 'label', 'idx'],
    num_rows: 5463
})


Dataset: test
Dataset({
    features: ['question', 'sentence', 'label', 'idx'],
    num_rows: 5463
})




In [ ]:
# print(dataset1['train'][''])

In [ ]:
# class CommonGenDataset(Dataset):
#     def __init__(self, split="train") -> None:
#         super().__init__()
#         assert split in ["train", "validation", "test"]
#         # data_df = load_dataset("allenai/common_gen", split=split, cache_dir="./cache/").to_pandas().groupby("concept_set_idx")

#         data_df = load_dataset("hugcyp/LCSTS", split=split, cache_dir="./cache/").to_pandas()
#         self.data = []
#         # for each in data_df:
#         #     targets = "/ ".join([s+"." if not s.endswith(".") else s for s in each[1].target.to_list()])
#         #     concepts = ", ".join(each[1].concepts.to_list()[0])
#         #     self.data.append({"concepts": concepts, "targets": targets})
#         for num in range(0,len(data_df)):
#           self.data.append({"summary":data_df['summary'][num],"text":data_df['text'][num]})


#     def __getitem__(self, index):
#         return self.data[index]

#     def __len__(self):
#         return len(self.data)



In [ ]:
# # 5. 前處理 IMDb 資料 (定義 function)
# def read_imdb_split(split_dir):
#     """針對 IMDb 資料集進行讀檔及正負向歸類
#     Args:
#         - split_dir: IMDb 資料集的資料夾路徑
#     Return:
#         - texts: 資料集的語句部分
#         - labels: 資料集的標籤部分
#     """
#     split_dir = Path(split_dir)
#     texts = []
#     labels = []
#     for label_dir in ["pos", "neg"]:
#         # 利用 iterdir() 來列出資料夾底下的所有檔案，此功能等同於 os.path.listdir()
#         # 使用 glob 的語法分取得副檔名為 .txt 的檔案
#         for text_file in (split_dir/label_dir).glob("*.txt"):
#             # read_text() 是 Pathlib 的好用功能
#             tmp_text = text_file.read_text()
#             # 將讀到的文字 append 到我們事先定義的 list 中
#             texts.append(tmp_text)
#             # 將資料夾標籤作為 label，並 append 到我們事先定義的 list 中
#             labels.append(0 if label_dir == "neg" else 1)

#     return texts, labels

In [ ]:
# # 6. 前處理 IMDb 資料 (執行)
# train_texts, train_labels = read_imdb_split('aclImdb/train')
# test_texts, test_labels = read_imdb_split('aclImdb/test')

In [ ]:
# print(train_texts[0])
# print(train_labels[0])

## Task 1: 資料載入
本次作業要求選擇GLUE benchmark中的資料集至少兩個\
以上是一種讀取資料集的範例，但具體怎麼讀不做要求，可以直接用huggingface的load_datasets

### 切分訓練資料，來分出 validation set

In [ ]:
# # 7. 使用 train_test_split 來切出 validation set

# from sklearn.model_selection import train_test_split

# # 設立隨機種子來控制隨機過程
# random_seed = 42

# # 設定要分出多少比例的 validation data
# valid_ratio = 0.2

# # 使用 train_test_split 來切分資料
# train_texts, val_texts, train_labels, val_labels = train_test_split(
#     train_texts,
#     train_labels,
#     test_size=valid_ratio,
#     random_state=random_seed
# )

## Hugging Face AutoTokenizer
- 使用 AutoTokenizer 搭配 Hugging Face models 的名稱可以直接呼叫使用
- 舉例:
    - transformers.AutoTokenizer.from_pretrained('roberta-base')
- [點這裡來查看 Hugging Face models 的名稱](https://huggingface.co/transformers/pretrained_models.html)

In [ ]:
# 8. 載入 tokenizer

# 在 Hugging Face 套件中可使用 .from_pretrained() 的方法來導入預訓練模型
tokenizer = transformers.AutoTokenizer.from_pretrained('roberta-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# 9. 分別將3種資料 (train/valid/test) 做 tokenization
# truncation 代表依照 max_length 進行序列長度的裁切
# max_length 可以在 tokenizer 的 parameters 中進行設定
# 如果沒有指定 max_length，則依照所使用的模型的序列最大長度
# padding 為 True 表示會將序列長度補齊至該 batch 的最大長度 (欲知詳情請查看 source code)

train_encodings = tokenizer(dataset1['train']['sentence'], truncation=True, padding=True)
val_encodings = tokenizer(dataset1['validation']['sentence'], truncation=True, padding=True)
test_encodings = tokenizer(dataset1['test']['sentence'], truncation=True, padding=True)

In [ ]:
# 10. 查看 max_length

tokenizer.model_max_length

512

### 檢查 tokenization 後的結果
- 使用 Hugging Face tokenizer 進行 tokenization 後的結果是一個 dict
- 這個 dict 的 keys 包含 'input_ids' 和 'attention_mask'
- input_ids: 原本句子中的每個字詞被斷詞後轉換成字典的 ID
    - 注意!! tokenizer 小小的動作已經幫你完成了斷詞和 word to ID 的轉換
- attention_mask: tokenization 後句子中包含文字的部分為 1，padding 的部分為 0
    - 可以想像成模型需要把注意力放在有文字的位置

In [ ]:
# 12. 檢查 tokenization 後的結果

print(val_encodings.keys())
print(val_encodings.input_ids[0])
print(val_encodings.attention_mask[0])

dict_keys(['input_ids', 'attention_mask'])
[0, 405, 128, 29, 10, 18452, 8, 747, 7920, 3251, 479, 1437, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# # 13. 透過 PyTorch Dataset 來建立能夠進行方便資料存取的格式

# class IMDbDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels):
#         # Dataset class 的 parameters 放入我們 tokenization 後的資料以及資料的標籤
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         # 請注意 tokenization 後的資料是一個 dict
#         # 在此步驟將資料以及標籤都轉換為 PyTorch 的 tensors
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])

#         return item

#     def __len__(self):
#         # 回傳資料集的總數
#         return len(self.labels)

# train_dataset = IMDbDataset(train_encodings, train_labels)
# val_dataset = IMDbDataset(val_encodings, val_labels)
# test_dataset = IMDbDataset(test_encodings, test_labels)

In [ ]:
# 13. 透過 PyTorch Dataset 來建立能夠進行方便資料存取的格式

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        # Dataset class 的 parameters 放入我們 tokenization 後的資料以及資料的標籤
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # 請注意 tokenization 後的資料是一個 dict
        # 在此步驟將資料以及標籤都轉換為 PyTorch 的 tensors
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])

        return item

    def __len__(self):
        # 回傳資料集的總數
        return len(self.labels)

train_dataset = Dataset(train_encodings, dataset1['train']['label'])
val_dataset = Dataset(val_encodings, dataset1['validation']['label'])
test_dataset = Dataset(test_encodings, dataset1['test']['label'])

### 除了自己處理資料，你還可以使用 Hugging Face Datasets
- Hugging Face Datasets 已經幫你收錄了自然語言處理領域常見的資料集
- 直接呼叫 Datasets 並搭配下面幾個 cells 的語法，可省下不少時間
- 但前提是你要進行的任務資料集有被收錄在 Hugging Face Datasets

In [ ]:
# 14. 查看 Hugging Face Datasets 的資訊

datasets_list = datasets.list_datasets()

print("現在 Hugging Face Datasets 有 {} 個資料集可以使用".format(len(datasets_list)))
print("===============================================")
# print("所有的資料集如下: ")
# print(', '.join(dataset for dataset in datasets_list))

<ipython-input-26-95e02c46990e>:3: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  datasets_list = datasets.list_datasets()


現在 Hugging Face Datasets 有 141638 個資料集可以使用


In [ ]:
# # 15. 從 Hugging Face Datasets 載入資料並做資料切分

# # 載入 IMDb 的訓練資料集
# train = datasets.load_dataset("imdb", split="train")

# # 設立隨機種子來控制隨機過程
# random_seed = 42
# # 從 IMDb 的訓練資料集中切分出驗證資料集
# splits = train.train_test_split(
#     test_size=0.2,
#     seed=random_seed
# )
# train, valid = splits['train'], splits['test']

# # 載入 IMDb 的測試資料集
# test = datasets.load_dataset("imdb", split="test")

In [ ]:
# print(len(train))
# print(len(valid))
# print(len(test))

In [ ]:
# # 16. 將 Hugging Face Datasets 轉為 PyTorch Dataset 的封裝

# def to_torch_data(hug_dataset):
#     """將 Hugging Face Datasets 轉為 PyTorch Dataset
#     Args:
#         - hug_dataset: 從 Datasets 載入的資料集
#     Return:
#         - dataset: 已轉為 PyTorch Dataset 的資料集
#     """
#     dataset = hug_dataset.map(
#         lambda batch: tokenizer(
#             batch["text"],
#             truncation=True,
#             padding=True
#         ),
#         batched=True
#     )
#     dataset.set_format(
#         type='torch',
#         columns=[
#             'input_ids',
#             'attention_mask',
#             'label'
#         ]
#     )
#     return dataset

# train_dataset = to_torch_data(train)
# val_dataset = to_torch_data(valid)
# test_dataset = to_torch_data(test)

In [ ]:
import torch
from torch.utils.data import Dataset

class HFDatasetWrapper(Dataset):
    def __init__(self, hf_dataset):
        self.hf_dataset = hf_dataset

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        sample = self.hf_dataset[idx]
        # 在这里进行必要的数据转换，例如将文本编码为数字
        # 例如，假设文本特征名为'sentence'，标签特征名为'label'
        text = sample['sentence']
        label = sample['label']
        # 在这里将标签转换为PyTorch张量
        label = torch.tensor(label, dtype=torch.long)  # 假设标签是整数编码的
        return {'text': text, 'label': label}

# 使用Hugging Face的数据集加载数据集
# from datasets import load_dataset
# dataset = load_dataset("glue", "sst2")

# 使用自定义的PyTorch Dataset封装数据集
pytorch_dataset1 = HFDatasetWrapper(dataset1['train'])

# 可以像普通的PyTorch Dataset一样使用pytorch_dataset
# 例如，创建一个数据加载器
from torch.utils.data import DataLoader
dataloader = DataLoader(pytorch_dataset1, batch_size=32, shuffle=True)


## 使用 Hugging Face 的模型
- 在這個 API 盛行的世代，總是有人幫你設想周到
- [Hugging Face 的模型頁面連結](https://huggingface.co/models)
- 以 Roberta 為例，只要透過 AutoModel.from_pretrained("roberta-base")，就可以直接使用 RobertaModel
- 需要注意的是接下來你要做怎樣的下游任務訓練
- 同樣以 Roberta 為例，在原始論文中 Roberta 進行過以下的任務:
    - Sentence pair classification: MNLI/QQP/QNLI/MRPC/RTE/WNLI
        - 對應 `RobertaForSequenceClassification`
        - 使用雙句結合，並以分類的方式進行訓練
    - Semantic textual similarity: STS-B
        - `RobertaForSequenceClassification`
        - 使用雙句結合，並以迴歸的方式進行訓練
    - Single sentence classification: SST-2/CoLA
        - 對應 `RobertaForSequenceClassification`
        - 使用單句，並以迴歸的方式進行訓練
    - Question answering: SQuAD v1.1/v2.0
        - 對應 `RobertaForQuestionAnswering`
        - 使用雙句(問題+原文)，並透過答案在原文中的位置進行訓練
    - Named-entity recognition (slot filling): CoNLL-2003
        - 對應 `RobertaForTokenClassification`
        - 使用單句，並以分類的方式進行訓練
- 如果要進行的下游任務訓練不在 Hugging Face 已經建好的模型範圍，那就需要自己寫一個 model class:
    1. 繼承 torch.nn.Module
    2. 利用 super 來繼承所有親屬類別的實體屬性
    3. 定義欲使用的 pre-trained model
    4. 定義會使用到的層如 linear 或 Dropout 等
    5. 設計 forward function 並且設定下游任務的輸出

In [ ]:
# 17.
# 利用 AutoModel 呼叫模型
model = transformers.AutoModelForSequenceClassification.from_pretrained("roberta-base")
# from transformers import BitFitForSequenceClassification, TrainingArguments, Trainer

# # 创建BitFit模型
# model = BitFitForSequenceClassification.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 進行模型的訓練
### 使用 Hugging Face Trainer ([Documentation](https://huggingface.co/transformers/main_classes/trainer.html))
- Trainer 是 Hugging Face 中高度封裝的套件之一，負責模型訓練時期的"流程"
- 過去我們自行寫訓練流程的程式碼可以交給 Trainer
- Trainer 需要搭配使用 [TrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments)
    - TrainingArguments 是 Trainer 所需要的引數

## Task2: 模型驗證
這裏要求同學們撰寫computer_metrics函式 \
要求同學們參考GLUE benchmark的官方網頁，使用和資料集對應的evaluation matrics\
回傳一個測試結果的dict

In [ ]:
# 18. 建立自定的評估的指標 (定義 function)
# 將作為 transformers.Trainer 的 parameters 之一

# Scikit-learn 的 precision_recall_fscore_support 套件可以一次計算 F1 score, precision, 和 recall
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# 請參考GLUE benchmark的官方網頁，使用和資料集對應的evaluation matrics

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = (preds == labels).mean()
    return {'accuracy': accuracy}

# 示例使用
# pred = {'predictions': [[0.9, 0.1], [0.3, 0.7], [0.8, 0.2]], 'label_ids': [0, 1, 0]}
# results = compute_metrics(pred)
# print(results)


## Task3: PEFT
在這個模塊中，我們要求同學們把模型改成PEFT的形式（Bitfit、LoRA或者其他）\
同學們可以將模型的可訓練參數量print出來

以下是各個資料集的baseline:

|dataset|metrics|baseline|
|----|----|----|
|CoLA|Matthew's Corr|0.6|
|SST2|Accuracy|0.88|
|MRPC|Accuracy|0.8|
|STSB|Pearson-Spearman Corr|0.8|
|QQP|F1 / Accuracy|0.8/0.8|
|MNLI_Matched|Accuracy|0.8|
|MNLI_Mismatched|Accuracy|0.8|
|QNLI|Accuracy|0.85|
|RTE|Accuracy|0.7|
|WNLI|Accuracy|0.8|

In [ ]:
# 19. 訓練模型

# 設定 TrainingArguments
training_args = transformers.TrainingArguments(
    output_dir="./results" ,          # 輸出的資料夾
    num_train_epochs= 3,              # 總共訓練的 epoch 數目
    learning_rate=2e-5 ,              # 學習率
    per_device_train_batch_size=8 ,  # 訓練模型時每個裝置的 batch size
    per_device_eval_batch_size=8 ,   # 驗證模型時每個裝置的 batch size
    gradient_accumulation_steps=2 ,   # 梯度累積的步數
    warmup_steps=500 ,                # learning rate scheduler 的參數
    weight_decay=0.01 ,               # 最佳化演算法 (optimizer) 中的權重衰退率
    evaluation_strategy= "steps",     # 設定驗證的時機
    save_strategy="epoch" ,           # 設定儲存的時機
    save_steps=500 ,                  # 設定多少步驟儲存一次模型
    eval_steps=500 ,                  # 設定多少步驟驗證一次模型
    report_to="tensorboard" ,         # 是否將訓練結果儲存到 TensorBoard
    save_total_limit= 1,              # 最多儲存幾個模型
    logging_dir="./logs" ,            # 存放 log 的資料夾
    logging_steps=10 ,
    seed=42 ,
)
# training_args = TrainingArguments(
#     output_dir=,          # 输出的文件夹
#     num_train_epochs=3,              # 总共训练的epoch数量
#     learning_rate=2e-5,              # 学习率
#     per_device_train_batch_size=8,   # 训练模型时每个设备的batch size
#     per_device_eval_batch_size=8,    # 验证模型时每个设备的batch size
#     warmup_steps=500,                # learning rate scheduler的参数
#     weight_decay=0.01,               # 最优化算法(optimizer)中的权重衰减率
#     evaluation_strategy="steps",     # 设置验证的时机
#     save_strategy="epoch",           # 设置保存的时机
#     logging_dir="./logs",            # 存放log的文件夹
#     logging_steps=10,
#     save_steps=500,                  # 设置多少步骤保存一次模型
#     eval_steps=500,                  # 设置多少步骤验证一次模型
# )

In [ ]:
trainer = transformers.Trainer(
    model=model,                         # 🤗 的模型
    args=training_args,                  # Trainer 所需要的引數
    train_dataset=train_dataset,         # 訓練集 (注意是 PyTorch Dataset)
    eval_dataset=val_dataset,            # 驗證集 (注意是 PyTorch Dataset)，可使 Trainer 在進行訓練時也進行驗證
    compute_metrics=compute_metrics,     # 自定的評估的指標
)

# 指定使用 1 個 GPU 進行訓練
trainer.args._n_gpu=1

# 開始進行模型訓練
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,0.371000,0.210876,0.917431
1000,0.288800,0.196461,0.930046
1500,0.205300,0.200901,0.927752
2000,0.313900,0.272511,0.915138
2500,0.225500,0.215138,0.934633
3000,0.324300,0.239541,0.928899
3500,0.133600,0.214983,0.942661
4000,0.243100,0.212161,0.940367
4500,0.108200,0.296282,0.936927
5000,0.146700,0.314271,0.913991


TrainOutput(global_step=12627, training_loss=0.18680105690502202, metrics={'train_runtime': 3063.5446, 'train_samples_per_second': 65.952, 'train_steps_per_second': 4.122, 'total_flos': 6955865942774760.0, 'train_loss': 0.18680105690502202, 'epoch': 3.0})

In [ ]:
print(model)

In [ ]:
# 21. 測試模型


trainer.predict(test_dataset)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
